In [1]:
import os
import pickle
import functools
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Pool
from scipy.spatial.distance import cosine
from ph_likes.util import queries
from ph_likes.util import settings

/home/can/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Load Data

In [2]:
# tables
df_post = queries.get_post_table()

In [3]:
CF = np.load(os.path.join(settings.LOCAL_MODELS_PATH, 'CF.npy'))
# post_ids
post_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'post_ids2index.pkl'), 'rb'))
index2post_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2post_ids.pkl'), 'rb'))
#
user_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'user_ids2index.pkl'), 'rb'))
index2user_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2user_ids.pkl'), 'rb'))

# Get Recommendation

In [26]:
def get_item_similarity(cf, item_index_1, item_index_2):
    v1 = cf[:, item_index_1]
    v2 = cf[:, item_index_2]
#     return 1.0 - cosine(v1, v2)
#     return (v1 == v2).sum()
    return np.bitwise_and(v1, v2).sum()

In [30]:
USER_ID = settings.EXAMPLE_USER_ID
USER_ID_INDEX = user_ids2index[USER_ID]
USER_ID_INDEX

890

In [31]:
liked_item_indices = np.where(CF[USER_ID_INDEX, :])[0][0:1]
not_liked_item_indices = np.array(list(set(range(CF.shape[1])) - set(liked_item_indices)))
ITEM_SIMILARITIES_FOR_USER = dict()

In [32]:
for item_index in tqdm_notebook(not_liked_item_indices):
    avg_similarity = np.mean([get_item_similarity(CF, item_index, i) for i in liked_item_indices])
    ITEM_SIMILARITIES_FOR_USER[item_index] = avg_similarity

KeyboardInterrupt: 

In [33]:
ITEM_SIMILARITIES_FOR_USER

{0: 30.0,
 1: 18.0,
 2: 19.0,
 3: 16.0,
 4: 15.0,
 5: 20.0,
 6: 12.0,
 7: 12.0,
 8: 14.0,
 9: 12.0,
 10: 12.0,
 11: 12.0,
 13: 17.0,
 14: 19.0,
 15: 20.0,
 16: 17.0,
 17: 16.0,
 18: 11.0,
 19: 13.0,
 20: 10.0,
 21: 12.0,
 22: 9.0,
 23: 7.0,
 24: 10.0,
 25: 5.0,
 26: 6.0,
 27: 6.0,
 28: 6.0,
 29: 7.0,
 30: 21.0,
 31: 21.0,
 32: 16.0,
 33: 13.0,
 34: 14.0,
 35: 14.0,
 36: 18.0,
 37: 9.0,
 38: 9.0,
 39: 10.0,
 40: 13.0,
 41: 8.0,
 42: 8.0,
 43: 11.0,
 44: 5.0,
 45: 9.0,
 46: 6.0,
 47: 6.0,
 48: 9.0,
 49: 7.0,
 50: 5.0,
 51: 7.0,
 52: 10.0,
 53: 5.0,
 54: 6.0,
 55: 14.0,
 56: 14.0,
 57: 14.0,
 58: 13.0,
 59: 11.0,
 60: 13.0,
 61: 10.0,
 62: 6.0,
 63: 6.0,
 64: 8.0,
 65: 8.0,
 66: 10.0,
 67: 6.0,
 68: 7.0,
 69: 5.0,
 70: 8.0,
 71: 8.0,
 72: 3.0,
 73: 2.0,
 74: 2.0,
 75: 5.0,
 76: 3.0,
 77: 3.0,
 78: 2.0,
 79: 3.0,
 80: 2.0,
 81: 2.0,
 82: 2.0,
 83: 1.0,
 84: 2.0,
 85: 20.0,
 86: 13.0,
 87: 15.0,
 88: 83.0,
 89: 13.0,
 90: 10.0,
 91: 9.0,
 92: 11.0,
 93: 9.0,
 94: 8.0,
 95: 7.0,
 96: 7.0,
 9

In [19]:
df = pd.DataFrame(pd.Series(ITEM_SIMILARITIES_FOR_USER)).reset_index()
df.columns = ['item_index', 'score']
df.sort_values(by='score')

,item_index,score
51311,51312,749174.0
63134,63135,752623.0
4025,4026,757622.0
4066,4067,758125.0
11636,11637,758909.0
4084,4085,759495.0
49347,49348,760199.0
19650,19651,760218.0
4043,4044,760295.0
15399,15400,760634.0


In [22]:
CF[0] == CF[1])

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
# # multiprocessing
# def get_avg_similarity(item_index, liked_item_indices, cf):
#     avg_similarity = np.mean([get_item_similarity(cf, item_index, i) for i in liked_item_indices])
#     return avg_similarity
# p_list = not_liked_item_indices
# p_func = functools.partial(get_avg_similarity, liked_item_indices=liked_item_indices, cf=CF)
# with Pool() as p:
#     for avg_similarity in tqdm_notebook(p.imap(p_func, p_list), total=len(not_liked_item_indices)):
#         ITEM_SIMILARITIES_FOR_USER[item_index] = avg_similarity